In [19]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import wfdb
import time
import random
from sklearn.preprocessing import minmax_scale
import sys
import torch

In [21]:
#pip install wfdb
#!pip install torch

In [23]:
##experiment with seed, run, channels(sinoatrial (SA) node, left feature lead, septal)
##later experiment with channel averaging of physician diagnosis features of myocardial infarction
##(septal average: v1, v2 ; left anterior average: v5, v6; left wrist: avl)
seed_num = 39 #sys.argv[1]  ##required bash files
#run_num = #sys.argv[2]
channel_1 = 'v6'#sys.argv[3]
channel_2 = 'vz'#sys.argv[4]
channel_3 = 'ii'#sys.argv[5]

In [25]:
with open('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/RECORDS') as fp:  
    lines = fp.readlines()

In [27]:
##lines   ##read lines in dataset records.txt

In [29]:
##divide into healthy and diseased .dat and .hea lists saved in files
files_myocardial = []
files_healthy = []

In [31]:
for file in lines:
    file_path = './ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1] + '.hea'
    
    ##read header to determine class ##focus on myocardial infarction
    if 'Myocardial infarction' in open(file_path).read():
        files_myocardial.append(file)
        
    if 'Healthy control' in open(file_path).read():
        files_healthy.append(file)

In [33]:
#files_healthy  ##check with CONTROL
#files_myocardial

In [35]:
##shuffle data (cross-validation)
np.random.seed(int(seed_num))
np.random.shuffle(files_myocardial) 
np.random.shuffle(files_healthy)

In [37]:
##split to train and test, 80% train, 20% test
healthy_train = files_healthy[:int(0.8*len(files_healthy))]
healthy_val = files_healthy[int(0.8*len(files_healthy)):]
myocardial_train = files_myocardial[:int(0.8*len(files_myocardial))]
myocardial_val = files_myocardial[int(0.8*len(files_myocardial)):]

In [39]:
##save names of train and val files
h_train = pd.DataFrame(healthy_train)
h_val = pd.DataFrame(healthy_val)
m_train = pd.DataFrame(myocardial_train)
m_val = pd.DataFrame(myocardial_val)

In [41]:
h_train.to_csv("h_train_idx.csv")
m_train.to_csv("m_train_idx.csv")
h_val.to_csv("h_val_idx.csv")
m_val.to_csv("m_val_idx.csv")

In [43]:
##remove intersections
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [45]:
patient_ids_healthy_train = []
patient_ids_healthy_val = []
patient_ids_myocardial_train = []
patient_ids_myocardial_val = []
##extract first 10 letters in file path string, which is the patient id
for index in healthy_train:
    patient_ids_healthy_train.append(index[0:10])
for index in healthy_val:
    patient_ids_healthy_val.append(index[0:10])
for index in myocardial_train:
    patient_ids_myocardial_train.append(index[0:10])
for index in myocardial_val:
    patient_ids_myocardial_val.append(index[0:10])

In [47]:
#type(patient_ids_myocardial_train)  ##list

In [49]:
len(np.unique(patient_ids_myocardial_train)) ##80% of total 148 patients is 118  
#likely to have duplicates, so distribute intersection over train and validation sets
#they can have common ecgs, used for checking

140

In [51]:
len(np.unique(patient_ids_myocardial_val)) ##20% of total 148 patients is 30

57

In [53]:
#patient_ids_myocardial_val ##stable

In [55]:
intersection_myocardial = intersection(patient_ids_myocardial_train, patient_ids_myocardial_val)
intersection_healthy = intersection(patient_ids_healthy_train, patient_ids_healthy_val)

In [57]:
##myocardial ##move half intersection to myocardial 
move_to_train = intersection_myocardial[:int(0.5*len(intersection_myocardial))]
move_to_val = intersection_myocardial[int(0.5*len(intersection_myocardial)):]

In [59]:
#move_to_train[0:5]  ##stable

In [61]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in myocardial_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            myocardial_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        myocardial_train.append(file_)

for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in myocardial_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            myocardial_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        myocardial_val.append(file_)

In [63]:
##healthy
move_to_train = intersection_healthy[:int(0.5*len(intersection_healthy))]
move_to_val = intersection_healthy[int(0.5*len(intersection_healthy)):]
#print(move_to_train[0:5]) ##stable

In [65]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in healthy_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            healthy_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        healthy_train.append(file_)

for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in healthy_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            healthy_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        healthy_val.append(file_)

In [67]:
len(np.unique(myocardial_val))

97

In [69]:
healthy_train[0][:-1]  ##example file directory

'patient240/s0468_re'

In [71]:
##update index files
h_train = pd.DataFrame(healthy_train)
h_val = pd.DataFrame(healthy_val)
m_train = pd.DataFrame(myocardial_train)
m_val = pd.DataFrame(myocardial_val)
h_train.to_csv("h_train_idx.csv")
m_train.to_csv("m_train_idx.csv")
h_val.to_csv("h_val_idx.csv")
m_val.to_csv("m_val_idx.csv")

In [73]:
##all files extracted and cleaned successfully
##now prepare a dataframe for the 3 channels
##Index----II----V6----VZ
data_healthy_train = []
for file in healthy_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_train.append(data)

In [74]:
data_healthy_train[0][:]

[array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855]),
 array([ 0.458 ,  0.458 ,  0.457 , ..., -0.335 , -0.3325, -0.329 ]),
 array([-0.1025, -0.1   , -0.0925, ...,  0.062 ,  0.0695,  0.065 ])]

In [75]:
healthy_train_set = pd.DataFrame(data_healthy_train, columns=['ii', 'v6', 'vz'] )

In [76]:
healthy_train_set.at[0, 'ii'] ##input the first healthy patient in training

array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855])

In [77]:
healthy_train_set.to_csv("healthy_train_signals.csv")

In [78]:
healthy_train_set.index  ##test for iterating over rows

RangeIndex(start=0, stop=60, step=1)

In [79]:
data_healthy_val = []
for file in healthy_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_val.append(data)

healthy_val_set = pd.DataFrame(data_healthy_train, columns=['ii', 'v6', 'vz'] )
healthy_val_set.to_csv("healthy_val_signals.csv")

In [80]:
data_unhealthy_train = []
for file in myocardial_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_train.append(data)

data_unhealthy_val = []
for file in myocardial_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_val.append(data)

unhealthy_train_set = pd.DataFrame(data_unhealthy_train, columns=['ii', 'v6', 'vz'] )
unhealthy_train_set.to_csv("mi_train_signals.csv")
unhealthy_val_set = pd.DataFrame(data_unhealthy_val, columns=['ii', 'v6', 'vz'] )
unhealthy_val_set.to_csv("mi_val_signals.csv")

In [88]:
##########DEBUGGING: TEST GENERATION OF SLICES
#unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_train))), k= 5)
#print(unhealthy_indices)
#unhealthy_batch = []
#for i in unhealthy_indices:
#    unhealthy_batch.append(data_unhealthy_train[i])

#print(unhealthy_batch[0][0])
#print(unhealthy_batch[0][1])
#print(unhealthy_batch[0][2])
##########DEBUGGING: TEST GENERATION OF SLICES

[73, 69, 55, 260, 177]
[-0.445  -0.445  -0.443  ...  0.0925  0.096   0.097 ]
[-0.201  -0.1885 -0.1905 ... -0.1065 -0.108  -0.1095]
[-1.29   -1.2855 -1.2905 ...  0.3335  0.341   0.336 ]


In [82]:
##############COMPONENT ONE IN .PY FILES##############
##normalize into batches, batch x and batch y (independent and dependent)
##minmax norm
window_size = 10000  ##from each ecg reading
def get_batch(batch_size, split='train'):
    ##random sampling of batches
    ##divide batch number in half to improve speed of network
    if split == 'train':
        ##random samples from sorted per batch with size k = batch_size / 2
        ##mimic shuffling
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_train))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_train))), k=int(batch_size / 2))
        unhealthy_batch = []
        healthy_batch = []
        for i in unhealthy_indices:
            unhealthy_batch.append(data_unhealthy_train[i])
        for j in healthy_indices:
            healthy_batch.append(data_healthy_train[j])
    elif split == 'val': 
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_val))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_val))), k=int(batch_size / 2))
        unhealthy_batch = []
        healthy_batch = []
        for i in unhealthy_indices:
            unhealthy_batch.append(data_unhealthy_val[i])
        for j in healthy_indices:
            healthy_batch.append(data_healthy_val[j])

    
    batch_x = []  ##batch of mixed healthy and unhealthy data
    for sample in unhealthy_batch: ##if val or if train
        start = random.choice(np.arange(len(sample[0]) - window_size))  ##randomly sample window from ecg 
        # normalize ecg values 
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
        
    for sample in healthy_batch:
        start = random.choice(np.arange(len(sample[0]) - window_size))
        # normalize
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
    
    batch_y = [0.1 for _ in range(int(batch_size / 2))]
    for _ in range(int(batch_size / 2)):
        batch_y.append(0.9)
        
    indices = np.arange(len(batch_y))
    np.random.shuffle(indices)
    
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    
    batch_x = batch_x[indices]
    batch_y = batch_y[indices]
    
    batch_x = np.reshape(batch_x, (-1, 3, window_size))
    batch_x = torch.from_numpy(batch_x)
    batch_x = batch_x.float()#.cuda()
    batch_x = batch_x.float()
    
    batch_y = np.reshape(batch_y, (-1, 1))
    batch_y = torch.from_numpy(batch_y)  ##save tensors
    batch_y = batch_y.float()#.cuda()
    batch_y = batch_y.float()
    
    return batch_x, batch_y

In [95]:
##constant batch size = 16  ##due to images ##or can be 10 
##test given train
##batch_x is the normalized windows of ecg channels in both healthy and unhealthy
batch_size = 10
#batch_x, batch_y = get_batch(batch_size, split='train') 

In [99]:
##save batch tensors in .txt for ready training and testing
##assume 80 training batches and 20 testing batches ##each batch has a size of 5 or 8
torch.save(batch_x, 'batch_x.pt')  ##encode in utf_8
torch.save(batch_y, 'batch_y.pt')  ##encode in utf_8

In [100]:
##in model, import the get_batches from this module then start
torch.load('batch_x.pt')

tensor([[[0.2105, 0.2093, 0.2101,  ..., 0.1412, 0.1399, 0.1452],
         [0.5839, 0.5845, 0.5839,  ..., 0.5336, 0.5342, 0.5336],
         [0.3101, 0.3105, 0.3116,  ..., 0.1840, 0.1840, 0.1867]],

        [[0.2089, 0.2042, 0.2042,  ..., 0.0450, 0.0454, 0.0399],
         [0.3956, 0.3941, 0.3926,  ..., 0.3554, 0.3521, 0.3475],
         [0.4373, 0.4537, 0.4638,  ..., 0.2777, 0.2671, 0.2715]],

        [[0.2367, 0.2376, 0.2363,  ..., 0.2185, 0.2168, 0.2222],
         [0.2886, 0.2895, 0.2886,  ..., 0.3931, 0.3941, 0.3945],
         [0.4784, 0.4683, 0.4656,  ..., 0.3410, 0.3282, 0.3322]],

        ...,

        [[0.4326, 0.4326, 0.4351,  ..., 0.0783, 0.0802, 0.0727],
         [0.5226, 0.5202, 0.5240,  ..., 0.5753, 0.5672, 0.5515],
         [0.0419, 0.0443, 0.0445,  ..., 0.8134, 0.8141, 0.8136]],

        [[0.0107, 0.0093, 0.0073,  ..., 0.0794, 0.0788, 0.0774],
         [0.4003, 0.3981, 0.3981,  ..., 0.4753, 0.4727, 0.4710],
         [0.4146, 0.4155, 0.4155,  ..., 0.5292, 0.5288, 0.5281]],

 

In [101]:
torch.load('batch_y.pt')

tensor([[0.9000],
        [0.9000],
        [0.1000],
        [0.9000],
        [0.1000],
        [0.9000],
        [0.1000],
        [0.1000],
        [0.1000],
        [0.9000]])